In [11]:
import numpy as np
import pandas as pd

In [12]:
import keras
keras.__version__

'2.3.1'

# Data Split

In [13]:
import random    
seed_num = 48
random.seed(seed_num)

x = np.load('/project/LSH/x_(7727,10,4069).npy')
y = np.load('/project/LSH/y_(7727,1).npy')

idx = list(range(len(x)))
random.shuffle(idx)

i = round(x.shape[0]*0.8)
X_train, y_train = x[idx[:i],:,:], y[idx[:i]]
X_test, y_test = x[idx[i:],:,:], y[idx[i:]]

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((6182, 10, 4069), (6182,), (1545, 10, 4069), (1545,))

# Revised KerasClassifier
- https://github.com/veniversum/keras/blob/9a401eb2e184fda7238a6259c1b8b02c645e4e9c/keras/wrappers/scikit_learn.py

In [14]:
import sklearn
import sklearn.utils

In [15]:
"""Wrapper for using the Scikit-Learn API with Keras models.
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import copy
import types

import numpy as np
import sklearn
from tensorflow import keras
from keras.utils.np_utils import to_categorical
from keras.utils.generic_utils import has_arg
from keras.models import Sequential
from keras.wrappers.scikit_learn import BaseWrapper

class BaseWrapper(object):
    """Base class for the Keras scikit-learn wrapper.

    Warning: This class should not be used directly.
    Use descendant classes instead.

    # Arguments
        build_fn: callable function or class instance
        **sk_params: model parameters & fitting parameters

    The `build_fn` should construct, compile and return a Keras model, which
    will then be used to fit/predict. One of the following
    three values could be passed to `build_fn`:
    1. A function
    2. An instance of a class that implements the `__call__` method
    3. None. This means you implement a class that inherits from either
    `KerasClassifier` or `KerasRegressor`. The `__call__` method of the
    present class will then be treated as the default `build_fn`.

    `sk_params` takes both model parameters and fitting parameters. Legal model
    parameters are the arguments of `build_fn`. Note that like all other
    estimators in scikit-learn, `build_fn` should provide default values for
    its arguments, so that you could create the estimator without passing any
    values to `sk_params`.

    `sk_params` could also accept parameters for calling `fit`, `predict`,
    `predict_proba`, and `score` methods (e.g., `epochs`, `batch_size`).
    fitting (predicting) parameters are selected in the following order:

    1. Values passed to the dictionary arguments of
    `fit`, `predict`, `predict_proba`, and `score` methods
    2. Values passed to `sk_params`
    3. The default values of the `keras.models.Sequential`
    `fit`, `predict`, `predict_proba` and `score` methods

    When using scikit-learn's `grid_search` API, legal tunable parameters are
    those you could pass to `sk_params`, including fitting parameters.
    In other words, you could use `grid_search` to search for the best
    `batch_size` or `epochs` as well as the model parameters.
    """

    def __init__(self, build_fn=None, **sk_params):
        self.build_fn = build_fn
        self.sk_params = sk_params
        self.check_params(sk_params)

    def check_params(self, params):
        """Checks for user typos in `params`.

        # Arguments
            params: dictionary; the parameters to be checked

        # Raises
            ValueError: if any member of `params` is not a valid argument.
        """
        legal_params_fns = [Sequential.fit, Sequential.predict,
                            Sequential.predict_classes, Sequential.evaluate]
        if self.build_fn is None:
            legal_params_fns.append(self.__call__)
        elif (not isinstance(self.build_fn, types.FunctionType) and
              not isinstance(self.build_fn, types.MethodType)):
            legal_params_fns.append(self.build_fn.__call__)
        else:
            legal_params_fns.append(self.build_fn)

        for params_name in params:
            for fn in legal_params_fns:
                if has_arg(fn, params_name):
                    break
            else:
                if params_name != 'nb_epoch':
                    raise ValueError(
                        '{} is not a legal parameter'.format(params_name))

    def get_params(self, **params):
        """Gets parameters for this estimator.

        # Arguments
            **params: ignored (exists for API compatibility).

        # Returns
            Dictionary of parameter names mapped to their values.
        """
        res = copy.deepcopy(self.sk_params)
        res.update({'build_fn': self.build_fn})
        return res

    def set_params(self, **params):
        """Sets the parameters of this estimator.

        # Arguments
            **params: Dictionary of parameter names mapped to their values.

        # Returns
            self
        """
        self.check_params(params)
        self.sk_params.update(params)
        return self

    def fit(self, x, y, **kwargs):
        """Constructs a new model with `build_fn` & fit the model to `(x, y)`.

        # Arguments
            x : array-like, shape `(n_samples, n_features)`
                Training samples where `n_samples` is the number of samples
                and `n_features` is the number of features.
            y : array-like, shape `(n_samples,)` or `(n_samples, n_outputs)`
                True labels for `x`.
            **kwargs: dictionary arguments
                Legal arguments are the arguments of `Sequential.fit`

        # Returns
            history : object
                details about the training history at each epoch.
        """
        if self.build_fn is None:
            self.model = self.__call__(**self.filter_sk_params(self.__call__))
        elif (not isinstance(self.build_fn, types.FunctionType) and
              not isinstance(self.build_fn, types.MethodType)):
            self.model = self.build_fn(
                **self.filter_sk_params(self.build_fn.__call__))
        else:
            self.model = self.build_fn(**self.filter_sk_params(self.build_fn))

        loss_name = self.model.loss
        if hasattr(loss_name, '__name__'):
            loss_name = loss_name.__name__
        if loss_name == 'categorical_crossentropy' and len(y.shape) != 2:
            y = to_categorical(y)

        fit_args = copy.deepcopy(self.filter_sk_params(Sequential.fit))
        fit_args.update(kwargs)

        history = self.model.fit(x, y, **fit_args)

        return history

    def filter_sk_params(self, fn, override=None):
        """Filters `sk_params` and returns those in `fn`'s arguments.

        # Arguments
            fn : arbitrary function
            override: dictionary, values to override `sk_params`

        # Returns
            res : dictionary containing variables
                in both `sk_params` and `fn`'s arguments.
        """
        override = override or {}
        res = {}
        for name, value in self.sk_params.items():
            if has_arg(fn, name):
                res.update({name: value})
        res.update(override)
        return res


class KerasClassifier(BaseWrapper):
    """Implementation of the scikit-learn classifier API for Keras.
    """

    def fit(self, x, y, sample_weight=None, **kwargs):
        """Constructs a new model with `build_fn` & fit the model to `(x, y)`.

        # Arguments
            x : array-like, shape `(n_samples, n_features)`
                Training samples where `n_samples` is the number of samples
                and `n_features` is the number of features.
            y : array-like, shape `(n_samples,)` or `(n_samples, n_outputs)`
                True labels for `x`.
            **kwargs: dictionary arguments
                Legal arguments are the arguments of `Sequential.fit`

        # Returns
            history : object
                details about the training history at each epoch.

        # Raises
            ValueError: In case of invalid shape for `y` argument.
        """
        y = np.array(y)
        if len(y.shape) == 2 and y.shape[1] > 1:
            self.classes_ = np.arange(y.shape[1])
        elif (len(y.shape) == 2 and y.shape[1] == 1) or len(y.shape) == 1:
            self.classes_ = np.unique(y)
            y = np.searchsorted(self.classes_, y)
        else:
            raise ValueError('Invalid shape for y: ' + str(y.shape))
        self.n_classes_ = len(self.classes_)
        if sample_weight is not None:
            kwargs['sample_weight'] = sample_weight
        return super(KerasClassifier, self).fit(x, y, **kwargs)

    def predict(self, x, **kwargs):
        """Returns the class predictions for the given test data.

        # Arguments
            x: array-like, shape `(n_samples, n_features)`
                Test samples where `n_samples` is the number of samples
                and `n_features` is the number of features.
            **kwargs: dictionary arguments
                Legal arguments are the arguments
                of `Sequential.predict_classes`.

        # Returns
            preds: array-like, shape `(n_samples,)`
                Class predictions.
        """
        kwargs = self.filter_sk_params(Sequential.predict_classes, kwargs)
        classes = self.model.predict_classes(x, **kwargs)
        return self.classes_[classes]

    def predict_proba(self, x, **kwargs):
        """Returns class probability estimates for the given test data.

        # Arguments
            x: array-like, shape `(n_samples, n_features)`
                Test samples where `n_samples` is the number of samples
                and `n_features` is the number of features.
            **kwargs: dictionary arguments
                Legal arguments are the arguments
                of `Sequential.predict_classes`.

        # Returns
            proba: array-like, shape `(n_samples, n_outputs)`
                Class probability estimates.
                In the case of binary classification,
                to match the scikit-learn API,
                will return an array of shape `(n_samples, 2)`
                (instead of `(n_sample, 1)` as in Keras).
        """
        kwargs = self.filter_sk_params(Sequential.predict_proba, kwargs)
        probs = self.model.predict_proba(x, **kwargs)

        # check if binary classification
        if probs.shape[1] == 1:
            # first column is probability of class 0 and second is of class 1
            probs = np.hstack([1 - probs, probs])
        return probs

    def score(self, x, y, **kwargs):
        """Returns the mean accuracy on the given test data and labels.

        # Arguments
            x: array-like, shape `(n_samples, n_features)`
                Test samples where `n_samples` is the number of samples
                and `n_features` is the number of features.
            y: array-like, shape `(n_samples,)` or `(n_samples, n_outputs)`
                True labels for `x`.
            **kwargs: dictionary arguments
                Legal arguments are the arguments of `Sequential.evaluate`.

        # Returns
            score: float
                Mean accuracy of predictions on `x` wrt. `y`.

        # Raises
            ValueError: If the underlying model isn't configured to
                compute accuracy. You should pass `metrics=["accuracy"]` to
                the `.compile()` method of the model.
        """
        y = np.searchsorted(self.classes_, y)
        kwargs = self.filter_sk_params(Sequential.evaluate, kwargs)

        loss_name = self.model.loss
        if hasattr(loss_name, '__name__'):
            loss_name = loss_name.__name__
        if loss_name == 'categorical_crossentropy' and len(y.shape) != 2:
            y = to_categorical(y)

        outputs = self.model.evaluate(x, y, **kwargs)
        if not isinstance(outputs, list):
            outputs = [outputs]
        for name, output in zip(self.model.metrics_names, outputs):
            if name == 'acc':
                return output
        raise ValueError('The model is not configured to compute accuracy. '
                         'You should pass `metrics=["accuracy"]` to '
                         'the `model.compile()` method.')

# Apply AdaboostClassifier
## adaboost1, get_model

In [30]:
seed_num = 42

import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, GRU, Dropout, LSTM, InputLayer
from sklearn.ensemble import VotingClassifier, AdaBoostClassifier

from sklearn import metrics 
from tensorflow import keras
# from keras.wrappers.scikit_learn import KerasClassifier
print(tf.__version__)
print(keras.__version__)


def get_model(seed_num):
    tf.random.set_seed(seed_num)

    lstm = Sequential()
    lstm.add(InputLayer(input_shape=(X_train.shape[1],X_train.shape[2])))
    lstm.add(LSTM(units=128, activation='hard_sigmoid', return_sequences=True))
    lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
    lstm.add(Dropout(0.2))
    lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
    lstm.add(LSTM(units=32, activation='hard_sigmoid', return_sequences=False))
    lstm.add(Dropout(0.2))
    lstm.add(Dense(units=1, activation='sigmoid'))
    optimizer = keras.optimizers.Adam(learning_rate = 0.001, decay=0.001)
    lstm.compile(optimizer= optimizer, loss = "binary_crossentropy", metrics=['acc'], sample_weight_mode = 'temporal')
    return lstm

2.5.0
2.5.0


In [31]:
# adaboost
# GRU_Predictors = KerasClassifier(build_fn=lambda:lstm, epochs=20, batch_size=256)
lstm_Predictors = KerasClassifier(build_fn=lambda:get_model(seed_num), epochs=20, batch_size=256)
lstm_Predictors._estimator_type="classifier"
final_model = AdaBoostClassifier(lstm_Predictors, n_estimators=100, random_state=42, learning_rate=1.2)

In [33]:
%%time
final_model.fit(X_train,y_train, sample_weight=np.ones(X_train.shape[0])/X_train.shape[0])

Epoch 1/20
25/25 [==============================] - 5s 92ms/step - loss: 1.1015e-04 - acc: 0.5919
Epoch 2/20
25/25 [==============================] - 2s 92ms/step - loss: 1.1018e-04 - acc: 0.5950
Epoch 3/20
25/25 [==============================] - 2s 90ms/step - loss: 1.0948e-04 - acc: 0.5992
Epoch 4/20
25/25 [==============================] - 2s 90ms/step - loss: 1.0941e-04 - acc: 0.5958
Epoch 5/20
25/25 [==============================] - 2s 90ms/step - loss: 1.0933e-04 - acc: 0.5953
Epoch 6/20
25/25 [==============================] - 2s 95ms/step - loss: 1.0905e-04 - acc: 0.6043
Epoch 7/20
25/25 [==============================] - 2s 95ms/step - loss: 1.0952e-04 - acc: 0.5982
Epoch 8/20
25/25 [==============================] - 2s 93ms/step - loss: 1.0943e-04 - acc: 0.6032
Epoch 9/20
10/25 [===========>..................] - ETA: 1s - loss: 1.0966e-04 - acc: 0.5984

KeyboardInterrupt: 

In [17]:
preds = final_model.predict(X_test)
preds[preds>0.5] = 1
preds[preds<=0.5] = 0
from sklearn import metrics
print('정확도 :', metrics.accuracy_score(y_test, preds))

AttributeError: 'AdaBoostClassifier' object has no attribute 'n_classes_'

In [ ]:
final_model.predict(X_test)

## adaboost 2
- 참고한 github
- https://github.com/limitless083/timeseries-forecast

In [34]:
import numpy as np


def calc_error(y, y_):
    return np.sqrt((y - y_) ** 2)


class AdaBoost:
    def __init__(self, trainX, trainY):
        self.trainX = trainX
        self.trainY = trainY
        self.N = len(self.trainX)
        self.weights = np.ones(self.N) / self.N
        self.alphas = []
        self.models = []

    def set_rule(self, model):
        predict = model.predict(self.trainX)
        errors = []
        for i in range(self.N):
            errors.append(self.weights[i] * calc_error(self.trainY[i], predict[i, 0]))
        e = np.sum(errors)
        alpha = 0.5 * np.log((1 - e) / e)
        print('e=%.4f a=%.4f' % (e, alpha))
        w = np.zeros(self.N)
        for i in range(self.N):
            w[i] = self.weights[i] * np.exp(alpha * errors[i] / e)
        self.weights = w / w.sum()
        self.models.append(model)
        self.alphas.append(alpha)

    def predict(self, x_set):
        n_models = len(self.models)
        alpha_sum = np.sum(self.alphas)
        final_predict = np.zeros(1)
        for i in range(n_models):
            predict = self.models[i].predict(x_set)
            final_predict = final_predict + predict[:, 0] * self.alphas[i]
        final_predict = final_predict / alpha_sum

        return final_predict.reshape(len(x_set), 1)

    def evaluate(self):
        n_models = len(self.models)
        alpha_sum = np.sum(self.alphas)
        final_predict = np.zeros(len(self.trainX))
        for i in range(n_models):
            predict = self.models[i].predict(self.trainX)
            final_predict = final_predict + predict[:, 0] * self.alphas[i]
        final_predict = final_predict / alpha_sum
        errors = []
        for i in range(self.N):
            errors.append(calc_error(self.trainY[i], final_predict[i]))
        return np.sum(errors)

    def get_weights(self):
        return self.weights

In [35]:
adaboost = AdaBoost(X_train, y_train)
for i in range(1):
    sample_weights = adaboost.get_weights()
    model = get_model(seed_num)
    model.fit(X_train, y_train, epochs=20, batch_size=156, verbose=2, sample_weight=sample_weights)
    adaboost.set_rule(model)
print("final error: ", adaboost.evaluate())

Epoch 1/20
40/40 - 5s - loss: 1.0949e-04 - acc: 0.5925
Epoch 2/20
40/40 - 3s - loss: 1.0935e-04 - acc: 0.5979
Epoch 3/20
40/40 - 3s - loss: 1.0992e-04 - acc: 0.5959
Epoch 4/20
40/40 - 3s - loss: 1.0989e-04 - acc: 0.6000
Epoch 5/20
40/40 - 3s - loss: 1.0933e-04 - acc: 0.5951
Epoch 6/20
40/40 - 3s - loss: 1.0938e-04 - acc: 0.5995
Epoch 7/20
40/40 - 3s - loss: 1.0936e-04 - acc: 0.6014
Epoch 8/20
40/40 - 3s - loss: 1.0958e-04 - acc: 0.6000
Epoch 9/20
40/40 - 3s - loss: 1.0910e-04 - acc: 0.6011
Epoch 10/20
40/40 - 3s - loss: 1.0926e-04 - acc: 0.6029
Epoch 11/20
40/40 - 3s - loss: 1.0906e-04 - acc: 0.6026
Epoch 12/20
40/40 - 3s - loss: 1.0922e-04 - acc: 0.6064
Epoch 13/20
40/40 - 3s - loss: 1.0903e-04 - acc: 0.6032
Epoch 14/20
40/40 - 3s - loss: 1.0909e-04 - acc: 0.6042
Epoch 15/20
40/40 - 3s - loss: 1.0904e-04 - acc: 0.6035
Epoch 16/20
40/40 - 3s - loss: 1.0910e-04 - acc: 0.6042
Epoch 17/20
40/40 - 3s - loss: 1.0886e-04 - acc: 0.6058
Epoch 18/20
40/40 - 3s - loss: 1.0883e-04 - acc: 0.6045
E

In [21]:
pred = model.predict(X_test)
pred[pred>0.5] = 1; pred[pred<=0.5] = 0
metrics.accuracy_score(y_test, pred)

0.6284789644012945

## adaboost 3
- 참고한 자료
- https://stackoverflow.com/questions/64558810/how-to-use-a-keras-model-inside-of-sklearns-adaboost

In [8]:
class MyKerasClassifier(KerasClassifier):
    def fit(self, x, y, sample_weight=None, **kwargs):
        y = np.array(y)
        if len(y.shape) == 2 and y.shape[1] > 1:
            self.classes_ = np.arange(y.shape[1])
        elif (len(y.shape) == 2 and y.shape[1] == 1) or len(y.shape) == 1:
            self.classes_ = np.unique(y)
            y = np.searchsorted(self.classes_, y)
        else:
            raise ValueError('Invalid shape for y: ' + str(y.shape))
        self.n_classes_ = len(self.classes_)
        if sample_weight is not None:
            kwargs['sample_weight'] = sample_weight
            print(type(sample_weight))
        return super(MyKerasClassifier, self).fit(x, y, **kwargs)
    def predict(self, x, **kwargs):
        kwargs = self.filter_sk_params(Sequential.predict_classes, kwargs)
        classes = self.model.predict_classes(x, **kwargs)
        return self.classes_[classes].flatten()
        #return super(KerasClassifier, self).fit(x, y, sample_weight=sample_weight)

In [11]:
base_estimator = MyKerasClassifier(build_fn=lambda:get_model(seed_num), epochs=20, batch_size=156, verbose=2)
boosted_classifier = AdaBoostClassifier(base_estimator=base_estimator,n_estimators=10,random_state=42)

boosted_classifier.fit(X_train, y_train)

<class 'numpy.ndarray'>
Epoch 1/20
40/40 - 16s - loss: 1.0959e-04 - acc: 0.5932
Epoch 2/20
40/40 - 3s - loss: 1.0909e-04 - acc: 0.6030
Epoch 3/20
40/40 - 3s - loss: 1.0925e-04 - acc: 0.5992
Epoch 4/20
40/40 - 3s - loss: 1.0919e-04 - acc: 0.6030
Epoch 5/20
40/40 - 3s - loss: 1.0931e-04 - acc: 0.5982
Epoch 6/20
40/40 - 3s - loss: 1.0895e-04 - acc: 0.6050
Epoch 7/20
40/40 - 3s - loss: 1.0938e-04 - acc: 0.6053
Epoch 8/20
40/40 - 3s - loss: 1.0933e-04 - acc: 0.6006
Epoch 9/20
40/40 - 3s - loss: 1.0891e-04 - acc: 0.6050
Epoch 10/20
40/40 - 3s - loss: 1.0880e-04 - acc: 0.6056
Epoch 11/20
40/40 - 3s - loss: 1.0915e-04 - acc: 0.6038
Epoch 12/20
40/40 - 3s - loss: 1.0866e-04 - acc: 0.6060
Epoch 13/20
40/40 - 3s - loss: 1.0878e-04 - acc: 0.6051
Epoch 14/20
40/40 - 3s - loss: 1.0874e-04 - acc: 0.6066
Epoch 15/20
40/40 - 3s - loss: 1.0881e-04 - acc: 0.6058
Epoch 16/20
40/40 - 3s - loss: 1.0876e-04 - acc: 0.6056
Epoch 17/20
40/40 - 3s - loss: 1.0860e-04 - acc: 0.6061
Epoch 18/20
40/40 - 3s - loss: 1

/usr/local/lib/python3.8/dist-packages/keras/engine/sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


40/40 - 2s
<class 'numpy.ndarray'>
Epoch 1/20
40/40 - 6s - loss: 1.1402e-04 - acc: 0.5325
Epoch 2/20
40/40 - 3s - loss: 1.1279e-04 - acc: 0.5209
Epoch 3/20
40/40 - 3s - loss: 1.1291e-04 - acc: 0.4982
Epoch 4/20
40/40 - 3s - loss: 1.1285e-04 - acc: 0.5045
Epoch 5/20
40/40 - 3s - loss: 1.1300e-04 - acc: 0.4995
Epoch 6/20
40/40 - 3s - loss: 1.1267e-04 - acc: 0.5055
Epoch 7/20
40/40 - 3s - loss: 1.1306e-04 - acc: 0.5152
Epoch 8/20
40/40 - 3s - loss: 1.1295e-04 - acc: 0.4934
Epoch 9/20
40/40 - 3s - loss: 1.1261e-04 - acc: 0.4913
Epoch 10/20
40/40 - 3s - loss: 1.1251e-04 - acc: 0.5333
Epoch 11/20
40/40 - 3s - loss: 1.1283e-04 - acc: 0.4948
Epoch 12/20
40/40 - 3s - loss: 1.1232e-04 - acc: 0.5210
Epoch 13/20
40/40 - 3s - loss: 1.1250e-04 - acc: 0.5160
Epoch 14/20
40/40 - 3s - loss: 1.1236e-04 - acc: 0.5134
Epoch 15/20
40/40 - 3s - loss: 1.1250e-04 - acc: 0.5070
Epoch 16/20


KeyboardInterrupt: 

In [ ]:
boosted_classifier.predict(X_test)

# Apply VotingClassifier

In [36]:
# voting
# GRU_Predictors = KerasClassifier(build_fn=lambda:lstm, epochs=20, batch_size=256)
# lstm_Predictors = KerasClassifier(build_fn=lambda:get_model(seed_num), epochs=20, batch_size=256)
#LSTM 쌓기
estimator = []
for i in range(1,11):
    LSTM_Predictors = KerasClassifier(build_fn=lambda:get_model(None), epochs=20, batch_size=256)
    LSTM_Predictors._estimator_type="classifier"
    estimator.append((f'model{i}', LSTM_Predictors))
print(estimator) 
voting_model = VotingClassifier(estimators=estimator, voting = 'soft')

[('model1', <__main__.KerasClassifier object at 0x7f2a1231eee0>), ('model2', <__main__.KerasClassifier object at 0x7f2a1231efd0>), ('model3', <__main__.KerasClassifier object at 0x7f2a1231e1c0>), ('model4', <__main__.KerasClassifier object at 0x7f2a1231e250>), ('model5', <__main__.KerasClassifier object at 0x7f2a1231e400>), ('model6', <__main__.KerasClassifier object at 0x7f2a1231eeb0>), ('model7', <__main__.KerasClassifier object at 0x7f2a1231e910>), ('model8', <__main__.KerasClassifier object at 0x7f2a12324190>), ('model9', <__main__.KerasClassifier object at 0x7f2a123248b0>), ('model10', <__main__.KerasClassifier object at 0x7f2a12324880>)]


In [37]:
%%time
voting_model.fit(X_train,y_train)

Epoch 1/20
25/25 [==============================] - 5s 89ms/step - loss: 0.6960 - acc: 0.5524
Epoch 2/20
25/25 [==============================] - 2s 87ms/step - loss: 0.6756 - acc: 0.5996
Epoch 3/20
25/25 [==============================] - 2s 88ms/step - loss: 0.6735 - acc: 0.5962
Epoch 4/20
25/25 [==============================] - 2s 86ms/step - loss: 0.6398 - acc: 0.6212
Epoch 5/20
25/25 [==============================] - 2s 89ms/step - loss: 0.5548 - acc: 0.7260
Epoch 6/20
25/25 [==============================] - 2s 89ms/step - loss: 0.4901 - acc: 0.7810
Epoch 7/20
25/25 [==============================] - 2s 90ms/step - loss: 0.4339 - acc: 0.8140
Epoch 8/20
25/25 [==============================] - 2s 90ms/step - loss: 0.3994 - acc: 0.8326
Epoch 9/20
25/25 [==============================] - 2s 89ms/step - loss: 0.3748 - acc: 0.8505
Epoch 10/20
25/25 [==============================] - 2s 90ms/step - loss: 0.3459 - acc: 0.8688
Epoch 11/20
25/25 [==============================] - 2s 90m

25/25 [==============================] - 2s 91ms/step - loss: 0.4151 - acc: 0.8213
Epoch 8/20
25/25 [==============================] - 2s 91ms/step - loss: 0.3837 - acc: 0.8442
Epoch 9/20
25/25 [==============================] - 2s 91ms/step - loss: 0.3642 - acc: 0.8505
Epoch 10/20
25/25 [==============================] - 2s 91ms/step - loss: 0.3366 - acc: 0.8714
Epoch 11/20
25/25 [==============================] - 2s 90ms/step - loss: 0.3229 - acc: 0.8801
Epoch 12/20
25/25 [==============================] - 2s 93ms/step - loss: 0.2991 - acc: 0.8932
Epoch 13/20
25/25 [==============================] - 2s 91ms/step - loss: 0.2816 - acc: 0.9015
Epoch 14/20
25/25 [==============================] - 2s 91ms/step - loss: 0.2895 - acc: 0.8926
Epoch 15/20
25/25 [==============================] - 2s 91ms/step - loss: 0.2657 - acc: 0.9088
Epoch 16/20
25/25 [==============================] - 2s 89ms/step - loss: 0.2431 - acc: 0.9191
Epoch 17/20
25/25 [==============================] - 2s 89ms/ste

25/25 [==============================] - 2s 91ms/step - loss: 0.3083 - acc: 0.8864
Epoch 14/20
25/25 [==============================] - 2s 91ms/step - loss: 0.2877 - acc: 0.8957
Epoch 15/20
25/25 [==============================] - 2s 93ms/step - loss: 0.2817 - acc: 0.8999
Epoch 16/20
25/25 [==============================] - 2s 89ms/step - loss: 0.2586 - acc: 0.9131
Epoch 17/20
25/25 [==============================] - 2s 88ms/step - loss: 0.2447 - acc: 0.9190
Epoch 18/20
25/25 [==============================] - 2s 88ms/step - loss: 0.2369 - acc: 0.9258
Epoch 19/20
25/25 [==============================] - 2s 88ms/step - loss: 0.2423 - acc: 0.9191
Epoch 20/20
25/25 [==============================] - 2s 88ms/step - loss: 0.2297 - acc: 0.9232
Epoch 1/20
25/25 [==============================] - 6s 92ms/step - loss: 0.7077 - acc: 0.5385
Epoch 2/20
25/25 [==============================] - 2s 92ms/step - loss: 0.6801 - acc: 0.5859
Epoch 3/20
25/25 [==============================] - 2s 89ms/step

VotingClassifier(estimators=[('model1',
                              <__main__.KerasClassifier object at 0x7f2a1231eee0>),
                             ('model2',
                              <__main__.KerasClassifier object at 0x7f2a1231efd0>),
                             ('model3',
                              <__main__.KerasClassifier object at 0x7f2a1231e1c0>),
                             ('model4',
                              <__main__.KerasClassifier object at 0x7f2a1231e250>),
                             ('model5',
                              <__main__.KerasClassifier object at 0x7f2a1231e400>),
                             ('model6',
                              <__main__.KerasClassifier object at 0x7f2a1231eeb0>),
                             ('model7',
                              <__main__.KerasClassifier object at 0x7f2a1231e910>),
                             ('model8',
                              <__main__.KerasClassifier object at 0x7f2a12324190>),
        

In [39]:
preds = voting_model.predict(X_test)
from sklearn import metrics
print('정확도 :', metrics.accuracy_score(y_test, preds))

/home/guri99/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:430: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


정확도 : 0.7715210355987056


# Adaboost-GRU

In [36]:
seed_num = 48
def get_gru_model(seed_num):
    tf.random.set_seed(seed_num)

    gru = Sequential()
    gru.add(InputLayer(input_shape=(X_train.shape[1],X_train.shape[2])))
    gru.add(GRU(units=128, activation='hard_sigmoid', return_sequences=True))
    gru.add(GRU(units=64, activation='hard_sigmoid', return_sequences=True))
    gru.add(Dropout(0.2))
    gru.add(GRU(units=64, activation='hard_sigmoid', return_sequences=True))
    gru.add(GRU(units=32, activation='hard_sigmoid', return_sequences=False))
    gru.add(Dropout(0.2))
    gru.add(Dense(units=1, activation='sigmoid'))

    gru.compile(optimizer= "adam", loss = "binary_crossentropy", metrics=['acc'])
    return gru

# adaboost
# GRU_Predictors = KerasClassifier(build_fn=lambda:gru, epochs=20, batch_size=256)
gru_Predictors = KerasClassifier(build_fn=lambda:get_gru_model(seed_num), epochs=20, batch_size=256)
gru_model = AdaBoostClassifier(gru_Predictors, n_estimators=10, random_state=42)

In [37]:
%%time
gru_model.fit(X_train,y_train)

Epoch 1/20
25/25 [==============================] - 5s 73ms/step - loss: 1.1196e-04 - acc: 0.5808
Epoch 2/20
25/25 [==============================] - 2s 74ms/step - loss: 1.1028e-04 - acc: 0.5911
Epoch 3/20
25/25 [==============================] - 2s 73ms/step - loss: 1.1036e-04 - acc: 0.5858
Epoch 4/20
25/25 [==============================] - 2s 72ms/step - loss: 1.1049e-04 - acc: 0.5904
Epoch 5/20
25/25 [==============================] - 2s 72ms/step - loss: 1.1094e-04 - acc: 0.5820
Epoch 6/20
25/25 [==============================] - 2s 75ms/step - loss: 1.0939e-04 - acc: 0.5893
Epoch 7/20
25/25 [==============================] - 2s 76ms/step - loss: 1.0990e-04 - acc: 0.5938
Epoch 8/20
25/25 [==============================] - 2s 76ms/step - loss: 1.1004e-04 - acc: 0.5930
Epoch 9/20
25/25 [==============================] - 2s 74ms/step - loss: 1.1012e-04 - acc: 0.5943
Epoch 10/20
25/25 [==============================] - 2s 76ms/step - loss: 1.0929e-04 - acc: 0.6065
Epoch 11/20
25/25 [

/usr/local/lib/python3.8/dist-packages/keras/engine/sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


Epoch 1/20
25/25 [==============================] - 5s 74ms/step - loss: 1.1916e-04 - acc: 0.5625
Epoch 2/20
25/25 [==============================] - 2s 73ms/step - loss: 1.1438e-04 - acc: 0.4882
Epoch 3/20
25/25 [==============================] - 2s 74ms/step - loss: 1.1424e-04 - acc: 0.5040
Epoch 4/20
25/25 [==============================] - 2s 75ms/step - loss: 1.1437e-04 - acc: 0.5045
Epoch 5/20
25/25 [==============================] - 2s 75ms/step - loss: 1.1482e-04 - acc: 0.4828
Epoch 6/20
25/25 [==============================] - 2s 76ms/step - loss: 1.1316e-04 - acc: 0.5082
Epoch 7/20
25/25 [==============================] - 2s 77ms/step - loss: 1.1378e-04 - acc: 0.5064
Epoch 8/20
25/25 [==============================] - 2s 75ms/step - loss: 1.1391e-04 - acc: 0.4710
Epoch 9/20
25/25 [==============================] - 2s 77ms/step - loss: 1.1379e-04 - acc: 0.4917
Epoch 10/20
25/25 [==============================] - 2s 75ms/step - loss: 1.1325e-04 - acc: 0.5208
Epoch 11/20
25/25 [

25/25 [==============================] - 2s 73ms/step - loss: 1.1379e-04 - acc: 0.5065
Epoch 8/20
25/25 [==============================] - 2s 76ms/step - loss: 1.1392e-04 - acc: 0.4683
Epoch 9/20
25/25 [==============================] - 2s 75ms/step - loss: 1.1380e-04 - acc: 0.4902
Epoch 10/20
25/25 [==============================] - 2s 76ms/step - loss: 1.1325e-04 - acc: 0.5185
Epoch 11/20
25/25 [==============================] - 2s 76ms/step - loss: 1.1289e-04 - acc: 0.5004
Epoch 12/20
25/25 [==============================] - 2s 75ms/step - loss: 1.1339e-04 - acc: 0.4845
Epoch 13/20
25/25 [==============================] - 2s 76ms/step - loss: 1.1284e-04 - acc: 0.5205
Epoch 14/20
25/25 [==============================] - 2s 76ms/step - loss: 1.1302e-04 - acc: 0.4956
Epoch 15/20
25/25 [==============================] - 2s 75ms/step - loss: 1.1335e-04 - acc: 0.5144
Epoch 16/20
25/25 [==============================] - 2s 73ms/step - loss: 1.1280e-04 - acc: 0.4956
Epoch 17/20
25/25 [=====

AdaBoostClassifier(base_estimator=<__main__.KerasClassifier object at 0x7f3a133592b0>,
                   n_estimators=10, random_state=42)

In [38]:
preds = gru_model.predict(X_test)
preds[preds>0.5] = 1
preds[preds<=0.5] = 0
from sklearn import metrics
print('정확도 :', metrics.accuracy_score(y_test, preds))

/usr/local/lib/python3.8/dist-packages/keras/engine/sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


정확도 : 0.6284789644012945


In [39]:
gru_model.predict(X_test)

/usr/local/lib/python3.8/dist-packages/keras/engine/sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


array([1, 1, 1, ..., 1, 1, 1])